read and process the video file

In [26]:
#! /usr/bin/python
import numpy as np
import cv2
cv2.destroyAllWindows()
# cap = cv2.VideoCapture('data/IMGP1653.MOV') # insect eye
# cap = cv2.VideoCapture('data/IMGP1421.MOV') # denim
# cap = cv2.VideoCapture('data/IMGP1652.MOV') # insect face
cap = cv2.VideoCapture('data/IMGP1661.MOV') # coin
# cap = cv2.VideoCapture('data/IMGP1659.MOV') # scrath on paper
# cap = cv2.VideoCapture('data/IMGP1654.MOV') # insect leg
stop_frame = 800
gradients = []
frames = []
iteration = 0
pause = False
break_loop = False

while not break_loop:
    if not pause:
        ret, frame = cap.read()
        if not ret:
            break
        if iteration == stop_frame:
            break
        iteration += 1
        # print(iteration)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.blur(gray, (3,3)) 
        laplacian = cv2.Laplacian(gray,cv2.CV_64F, ksize=3)
        # laplacian = cv2.Sobel(src=gray, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=3) # Combined X and Y Sobel Edge Detection
        # laplacian = cv2.Laplacian(gray,cv2.CV_64F)

        laplacian = cv2.convertScaleAbs( laplacian )

        gradients.append(laplacian)
        frames.append(frame)

        effective_image = np.vstack((laplacian, gray))

        width = int(effective_image.shape[1] * 0.4)
        height = int(effective_image.shape[0] * 0.4)
        dim = (width, height)
        cv2.imshow('window', cv2.resize(effective_image, dim, interpolation = cv2.INTER_AREA))
    
    k = cv2.waitKey(1)

    if k == ord('p'):
        pause = not pause
        print("pause", pause)
    if k == ord('q'):
        break_loop = True 

frames = np.array(frames)
frames = frames.transpose(1,2,0,3)

get the depth.

In [27]:
cv2.destroyAllWindows()
gradients = np.array(gradients)
maxima = np.amax(gradients, axis=0)



# maxima = cv2.blur(maxima, (3,3))

width = int(maxima.shape[1] * 0.4)
height = int(maxima.shape[0] * 0.4)
dim = (width, height)
indices = np.array(gradients.argmax(axis=0)).astype(np.uint32)
indices_blurred = cv2.blur(indices.astype(np.uint8), (13,13))
mxm = cv2.resize(maxima, dim, interpolation = cv2.INTER_AREA).astype(np.float)
cv2.imshow('gradient_maxima', mxm.astype(np.uint8))
mxm*=255.0/np.amax(mxm)
cv2.imshow('gradient_maxima_jet', cv2.applyColorMap(mxm.astype(np.uint8), cv2.COLORMAP_JET))
cv2.imshow('gradient_maxima_indices', cv2.resize(indices.astype(np.uint8), dim, interpolation = cv2.INTER_AREA))

cv2.waitKey(100)


# scatter_points = []

# scatter_np_points = np.array[]
# scatter_np_points = 


-1

the following section displays the result using open3d library

In [28]:

cv2.destroyAllWindows()
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.ndimage.filters import gaussian_filter

smoothing_kernel_size = 3
kernel = np.ones((smoothing_kernel_size, smoothing_kernel_size), np.uint8)

threshold =  np.percentile(maxima, 70)
print("threshold: ", threshold)
mask = np.array(maxima>threshold).astype(np.uint8)
# mask = cv2.erode(mask, kernel)
mask = cv2.dilate(mask, kernel)
# mask = cv2.dilate(mask, kernel)
# mask = cv2.erode(mask, kernel)
mask = cv2.erode(mask, kernel)
mask = cv2.erode(mask, kernel)
cv2.imshow("mask", mask*255)
cv2.waitKey(100)
negated_mask = np.ones(mask.shape)-mask
# indices_blurred = cv2.blur(indices.astype(np.uint8), (13,13))
indices_blurred = gaussian_filter(indices.astype(np.float), sigma=smoothing_kernel_size)
indices_blurred = indices_blurred * mask + indices * negated_mask

(r, c) = np.shape(indices)
# print (r,c,b)
(X, Y) = np.meshgrid(range(0, c), range(0, r))


#convert array of maximum gradient indices to a vector of 3d points
scatter_points = np.zeros((1080,1920,3))
scatter_points[:,:,0] = X
scatter_points[:,:,1] = Y
scatter_points[:,:,2] = indices_blurred*5
shp = scatter_points.shape
scatter_points = scatter_points.transpose(1,0,2)
scatter_points = scatter_points.reshape(1920*1080,3)
pcd = o3d.geometry.PointCloud()


pcd.points = o3d.utility.Vector3dVector(scatter_points)


b, c  = np.meshgrid(range(0,1080),range(0,1920))
colors = frames[b,c,indices[b,c],:].reshape(1920*1080,3)
colors = np.flip(colors, axis=1)
# colors = np.array(scatter_points)[:,3:6]
pcd.colors = o3d.utility.Vector3dVector(colors/256.0)
# cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
cl, ind = pcd.remove_radius_outlier(nb_points=16, radius=10)
# cl = cl.voxel_down_sample(voxel_size=10)

o3d.io.write_point_cloud("pointcloud.pcd", cl)

o3d.visualization.draw_geometries([cl])
# o3d.visualization.draw_geometries([pcd])
cv2.destroyAllWindows()

/tmp/ipykernel_101913/3594051229.py:6: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter
